# N-gram Language Modeling

작은 샘플들을 활용해 N-gram language modeling을 구현해보고, 구현한 N-gram language model로 자동 완성 기능을 이해해보자

In [1]:
corpus = ["오늘 날씨 어때",
          "오늘 날씨 어때",
          "오늘 날씨 어때",
          "오늘 축구 경기",
          "오늘 경기 결과",
          "오늘 경기 결과",
          "내일 날씨",
          "내일 축구 경기",
          "내일 축구 경기",
          "축구 일정",
          ]

# Tokenization

In [2]:
# tokenization
vocabs = set()
for sample in corpus:
    for token in sample.split():
        vocabs.add(token)

In [3]:
vocabs = list(vocabs)

In [4]:
vocabs

['결과', '경기', '어때', '일정', '축구', '오늘', '날씨', '내일']

In [5]:
idx_to_token = vocabs
token_to_idx = {token: idx for idx, token in enumerate(idx_to_token)}

In [6]:
token_to_idx

{'결과': 0, '경기': 1, '어때': 2, '일정': 3, '축구': 4, '오늘': 5, '날씨': 6, '내일': 7}

# Bi-gram language modeling

In [ ]:
# bi-gram language modeling은 P(next_token|previsous_token)을 사용하는 모델
# 우리는 P(previous token, next_token)의 joint probability distribution을 알 수 있으면
# 위의 P(next_token|previsous_token)를 계산하기 쉬움

In [ ]:
# 그럼 먼저 P(previous token, next_token)는 어떻게 구할까? 
# 먼저 카운팅! prvious_token -> next_token에 대한 테이블을 만들어서 카운팅을 수행한다.

In [7]:
prev_token_to_next_token_cnt_table = {}

In [8]:
total_cnt = 0

for sample in corpus:
    tokens = sample.split()

    for prev_token, next_token in zip(tokens[:-1], tokens[1:]):
        if prev_token not in prev_token_to_next_token_cnt_table:
            prev_token_to_next_token_cnt_table[prev_token] = {}
        if next_token not in prev_token_to_next_token_cnt_table[prev_token]:
            prev_token_to_next_token_cnt_table[prev_token][next_token] = 0
        prev_token_to_next_token_cnt_table[prev_token][next_token] += 1
        total_cnt += 1

In [9]:
prev_token_to_next_token_cnt_table

{'오늘': {'날씨': 3, '축구': 1, '경기': 2},
 '날씨': {'어때': 3},
 '축구': {'경기': 3, '일정': 1},
 '경기': {'결과': 2},
 '내일': {'날씨': 1, '축구': 2}}

In [10]:
import pandas as pd

In [14]:
prev_token_to_next_token_table_df = pd.DataFrame(prev_token_to_next_token_cnt_table).transpose()
prev_token_to_next_token_table_df.fillna(0,inplace=True)
prev_token_to_next_token_table_df
# 맨 왼쪽 컬럼이 prev_token, 첫번째 열이 next_token

,날씨,축구,경기,어때,일정,결과
오늘,3.0,1.0,2.0,0.0,0.0,0.0
날씨,0.0,0.0,0.0,3.0,0.0,0.0
축구,0.0,0.0,3.0,0.0,1.0,0.0
경기,0.0,0.0,0.0,0.0,0.0,2.0
내일,1.0,2.0,0.0,0.0,0.0,0.0


In [16]:
prev_token_to_next_token_join_prob_table_df = prev_token_to_next_token_table_df / total_cnt
marginal_prob = prev_token_to_next_token_join_prob_table_df.sum(axis=1)

In [17]:
marginal_prob # marginal probabilty -> P(prev_token)

오늘    0.333333
날씨    0.166667
축구    0.222222
경기    0.111111
내일    0.166667
dtype: float64

In [21]:
prev_token_to_next_token_cond_prob_table_df = prev_token_to_next_token_join_prob_table_df.copy()

In [22]:
for row_name, row in prev_token_to_next_token_join_prob_table_df.iterrows():
    for col_name, val in row.iteritems():
        prev_token_to_next_token_cond_prob_table_df.loc[row_name, col_name] /= marginal_prob[row_name]

prev_token_to_next_token_cond_prob_table_df        

,날씨,축구,경기,어때,일정,결과
오늘,0.500000,0.166667,0.333333,0.0,0.00,0.0
날씨,0.000000,0.000000,0.000000,1.0,0.00,0.0
축구,0.000000,0.000000,0.750000,0.0,0.25,0.0
경기,0.000000,0.000000,0.000000,0.0,0.00,1.0
내일,0.333333,0.666667,0.000000,0.0,0.00,0.0


## Bi-gram을 이용한 Naive한 자동 완성 기능

In [23]:
x = "오늘"

# split input
input_tokens = x.split()

if input_tokens:
    last_token = input_tokens[-1]

# lookup cond prob distribution
prev_token_to_next_token_cond_prob_table_df.loc[last_token]

날씨    0.500000
축구    0.166667
경기    0.333333
어때    0.000000
일정    0.000000
결과    0.000000
Name: 오늘, dtype: float64

In [24]:
def suggest(query, depth=2, top_k=2):
    # split input
    input_tokens = query.split()

    if input_tokens:
        last_token = input_tokens[-1]
    else:
        last_token = marginal_prob.sort_values(ascending=False).index[0]
    
    suggested_list = []
    
    def dfs(last_token, prev_token_list, depth):
        # terminal
        if depth <= 0:
            return
        
        # recursion
        next_token_candidates = prev_token_to_next_token_cond_prob_table_df.loc[last_token].sort_values(ascending=False)
        next_token_candidates = next_token_candidates[next_token_candidates > 0] # 확률이 0 이상인 것만
        
        last_token_list = next_token_candidates.keys().tolist()[:top_k]

        for last_token in last_token_list:
            new_suggest = prev_token_list + [last_token]
            suggested_list.append(new_suggest)
            
            if last_token in prev_token_to_next_token_cond_prob_table_df.index:
                dfs(last_token, prev_token_list=new_suggest, depth=depth-1)
            
    dfs(last_token, [last_token], depth)
    
    # concate tokens
    
    suggested_list = [ " ".join(tokens) for tokens in suggested_list]
    return suggested_list

In [25]:
suggest("축구", depth=2)

['축구 경기', '축구 경기 결과', '축구 일정']